In [1]:
import os
import re
import sys
import copy

ROOT = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from moz_sql_parser import parse as ps
sys.setrecursionlimit(1000000)
import json
import os
import pandas as pd
from termcolor import colored
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [2]:
from app.dataService.dataService import DataService

In [3]:
# Generate SQL and the execution results from the plain text
text = "films and film prices that cost below 10 dollars"
db_id = "cinema"
dataService = DataService("spider")

=== begin loading model ===
before load_trees
before connecting
before load_trees
before connecting


Some weights of the model checkpoint at Salesforce/grappa_large_jnt were not used when initializing RobertaModel: ['q_tab_dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias_col', 'lm_head.dense.weight', 'lm_head.decoder_col.weight', 'q_tab_dense.bias', 'lm_head.layer_norm_col.weight', 'lm_head.bias', 'lm_head.dense_col.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm_col.bias', 'lm_head.dense_col.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'amp': True, 'base_dim': 32, 'batch_size': 20, 'beam_encoder_num_layers': 1, 'cntx_beam': False, 'cntx_rep': False, 'cntx_reranker': True, 'debug': False, 'disentangle_cntx': True, 'embedding_dim': 256, 'grad_acum': 4, 'grad_clip': None, 'grad_norm': None, 'is_oracle': False, 'lin_after_cntx': False, 'lm_lr': 3e-06, 'load_less': False, 'lr': 0.000186, 'max_steps': 60000, 'num_heads': 8, 'power': 0.5, 'rat_dropout': 0.2, 'rat_layers': 8, 'should_rerank': False, 'temperature': 1, 'tfixup': False, 'tiny_dataset': False, 'train_as_dev': False, 'tree_rep_transformer_num_layers': 1, 'uniquify': False, 'use_bce': False, 'use_treelstm': False, 'utt_aug': True, 'value_pred': True}
=== finish loading model ===


In [22]:
# read spider dataset
dataset_type = "train_spider"
# dataset_type="dev"
with open(f"../backend/app/data/dataset/spider/{dataset_type}.json", "r") as f:
    spider_data = json.load(f)
df = pd.DataFrame(spider_data)

In [23]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


## parse sql templates

### 1. sql select

In [ ]:
split_symbol = " ; "
select_dict = {
    "phrases": {},
    "distinct": {},
    "count": {
        "phrases": {},
        "distinct": {}
    },
    "max": {},
    "min": {},
    "sum": {},
    "avg": {},
    "+": {},
    "-": {},
    "*": {},
    "/": {},
}

def decode_val_unit(val_unit, table):
    # (unit_op, col_unit1, col_unit2)
    table_names = table["table_names"]
    columns = table["column_names"]
    unit_op = dataService.global_variable.UNIT_OPS[val_unit[0]]
    col_unit1 = val_unit[1]
    col_unit1_decoded = decode_col_unit(col_unit1, table)
    col_unit2 = val_unit[2]
    col_unit2_decoded = decode_col_unit(col_unit2, table)
    return (unit_op, col_unit1_decoded, col_unit2_decoded)

def decode_col_unit(col_unit, table):
    if col_unit == None:
        return None
    else:
        agg_id = dataService.global_variable.AGG_OPS[col_unit[0]]
        col = table["column_names"][col_unit[1]]
        col_name = table["table_names"][col[0]] + " " + col[1]
        distinct_flag = "distinct" if col_unit[2] else ""
        return (agg_id, col_name, distinct_flag)

def decode_select(select, table):
    select = sql_label["select"]
    distinct_flag = "distinct" if select[0] else ""
    print([[distinct_flag, decode_val_unit(s[1], table)] for s in select[1]])

for rowid, row in df[["query", "db_id", "question"]].iterrows():
    if rowid != 3153:
        db_id = row["db_id"]
        sql = row["query"]
        parsed = dataService.parsesql(sql, db_id)
        
        sql_label = parsed["sql_parse"]
        table = parsed["table"]
        
        print(sql_label["from"])
        
        

In [41]:
# 'select': (isDistinct(bool), [(agg_id, val_unit), (agg_id, val_unit), ...])
# val_unit: (unit_op, col_unit1, col_unit2)
# col_unit: (agg_id, col_id, isDistinct(bool))

# 'from': {'table_units': [table_unit1, table_unit2, ...], 'conds': condition}
# table_unit: (table_type, col_unit/sql)

split_symbol = " ; "
select_dict = {
    "phrases": {},
    "distinct": {},
    "count": {
        "phrases": {},
        "distinct": {}
    },
    "max": {},
    "min": {},
    "sum": {},
    "avg": {},
    "+": {},
    "-": {},
    "*": {},
    "/": {},
}

for rowid, row in df[["query", "db_id", "question"]].iterrows():
    if rowid != 3153:
        db_id = row["db_id"]
        sql = row["query"]
        parsed = dataService.parsesql(sql, db_id)
        
        sql_label = parsed["sql_parse"]
        table = parsed["table"]
        table_names = table["table_names"]
        columns = table["column_names"]
        
        select = sql_label["select"]
        distinct_flag = select[0]
        
        if distinct_flag:
            distinct_select = split_symbol.join([table_names[columns[s[1][1][1]][0]] + " " + columns[s[1][1][1]][1] for s in select[1]])
            
        else:
            for s in select[1]:
                agg_id = dataService.global_variable.AGG_OPS[s[0]]
                val_unit = s[1]
                unit_ops = dataService.global_variable.UNIT_OPS[val_unit[0]]
                # --- col unit1: agg_id, col_id, isDistinct(bool)
                col_unit1 = val_unit[1]            
                col_agg_id1 = dataService.global_variable.AGG_OPS[col_unit1[0]]
                col1 = columns[col_unit1[1]]
                col_distinct_flag1 = col_unit1[2]
                table_id1 = col1[0]
                col_name1 = col1[1]
                select_name1 = table_names[table_id1] + " " + col_name1
                
                if unit_ops != "none":
                    # print(colored(agg_id, "red"), colored(unit_ops+"-"+col_agg_id1 + "-" + str(col_distinct_flag1), "blue") )
                    # print(sql)
                    # +;-;*;/
                    col_unit2 = val_unit[2]
                    col2 = columns[col_unit2[1]]
                    table_id2 = col2[0]
                    col_name2 = col2[1]
                    select_name2 = table_names[table_id2] + " " + col_name2
                    select_item = split_symbol.join([select_name1, select_name2])
                    if select_item not in select_dict[unit_ops].keys():
                        select_dict[unit_ops][select_item] = 0
                    select_dict[unit_ops][select_item] += 1
                
                else:
                    # print(val_unit[2], type(val_unit[2]))
                    # only one unit
                    if agg_id == "none":
                        if select_name1 not in select_dict["phrases"].keys():
                            select_dict["phrases"][select_name1] = 0
                        select_dict["phrases"][select_name1] += 1
                    elif agg_id == "count":
                        if col_distinct_flag1:
                            if select_name1 not in select_dict["count"]["distinct"].keys():
                                select_dict["count"]["distinct"][select_name1] = 0
                            select_dict["count"]["distinct"][select_name1] += 1
                        else:
                            if select_name1 not in select_dict["count"]["phrases"].keys():
                                select_dict["count"]["phrases"][select_name1] = 0
                            select_dict["count"]["phrases"][select_name1] += 1
                    else: # min, max, sum, avg
                        if select_name1 not in select_dict[agg_id].keys():
                            select_dict[agg_id][select_name1] = 0
                        select_dict[agg_id][select_name1] += 1
print()                  
                    

#         for s in select[1]:
#             agg_id = dataService.global_variable.AGG_OPS[s[0]]
#             val_unit = s[1]
#             unit_ops = dataService.global_variable.UNIT_OPS[val_unit[0]]
#             # --- col unit1: agg_id, col_id, isDistinct(bool)
#             col_unit1 = val_unit[1]            
#             col_agg_id1 = dataService.global_variable.AGG_OPS[col_unit1[0]]
#             col1 = columns[col_unit1[1]]
#             col_distinct_flag1 = col_unit1[2]
#             table_id1 = col1[0]
#             col_name1 = col1[1]
#             select_name1 = table_names[table_id1] + " " + col_name1
            
#             if col_distinct_flag1:
#                 print(colored(agg_id, "red"), colored(col_agg_id1, "blue"), sql)
            
#             if col_agg_id1 != "none":
#                 print(sql)
            
#             if unit_ops != "none":                
#                 # --- col unit2
#                 col_unit2 = val_unit[2]
#                 col2 = columns[col_unit2[1]]
#                 table_id2 = col2[0]
#                 col_name2 = col2[1]
#                 select_name2 = table_names[table_id2] + " " + col_name2
#                 select_item = split_symbol.join([select_name1, select_name2])
#                 if select_item not in select_dict[unit_ops].keys():
#                     select_dict[unit_ops][select_item] = 0
#                 select_dict[unit_ops][select_item] += 1
#             else:
#                 # one col_unit
#                 if distinct_flag:
#                     # --- distinct
#                     if len(select[1]) > 1:
#                         print(colored(sql, "yellow"))
#                 else:
#                     if col_agg_id1 != "none":
#                         print(colored(f"{agg_id}-{col_agg_id1} " + select_name1, "yellow"))
#


None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

...

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'NoneType'>
None <class 'Non

## sequence data mining & initial labeling

In [4]:
import random

max_seq = 10
min_seq = 2
min_sim_th = 0.2
max_sim_th = 0.75

def cos_sine_sim(sen0, sen1):
    embeddings1 = model.encode(sen0, convert_to_tensor=False)
    embeddings2 = model.encode(sen1, convert_to_tensor=False)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores

In [223]:
# generate sequence
import torch
from tqdm import tqdm
# print("randint: ", random.randint(2,4))

all_seqs_num = 0
all_seqs = {}

for name, group in tqdm(df.groupby("db_id")):
    # 1. NL query length
    nl_len = group.question_toks.agg(len)
    # 2. SQL length
    sql_len = group.query_toks_no_value.agg(len)
    group["nl_sql_len"] = (nl_len + sql_len)/2
    # 3. sort by sql_nl length
    group = group.sort_values('nl_sql_len', ascending=True)
    
    if len(group)>=2:
        sql_q = np.array([" ".join(v) for v in group.query_toks_no_value], dtype="object")
        nl_q = group.question.values
        
        selected = []
        nl_sq = []
        sql_sq = []
        for idx, (nl, sql) in enumerate(zip(nl_q, sql_q)):
            if idx not in selected:
                # natural language/sql similarity
                nl_sim = cos_sine_sim(nl, nl_q).numpy()[0]
                sql_sim  = cos_sine_sim(sql, sql_q).numpy()[0]
                # average similarity
                # mean_sim = (np.abs(nl_sim) + np.abs(sql_sim))/2
                mean_sim = nl_sim
                # print(f"NL sim: {nl_sim} \n SQL sim: {sql_sim} \n Mean sim: {mean_sim}")
                # sort out similarity value
                sort_idx = mean_sim.argsort()[::-1]
                sort_idx_filtered = [sid for sid in sort_idx if (mean_sim[sid] <= max_sim_th) and (mean_sim[sid]>min_sim_th) and (sid not in selected)]
                # print(mean_sim[sort_idx_filtered])
                if len(sort_idx_filtered)>=2:
                    rand_num = random.randint(min_seq, max_seq)
                    select_num = rand_num if len(sort_idx_filtered)>=rand_num else len(sort_idx_filtered)
                    # print(sql, nl_q[sort_idx_filtered[:select_num]])
                    # print(sql, sort_idx_filtered[:select_num], nl_q[sort_idx_filtered[:select_num]])
                    selected.extend(sort_idx_filtered[:select_num])
                    nl_sq.append(nl_q[sort_idx_filtered[:select_num]].tolist())
                    sql_sq.append(sql_q[sort_idx_filtered[:select_num]].tolist())
        all_seqs_num += len(selected_sq)
        if len(selected_sq)>0:
            all_seqs[name] = {}
            all_seqs[name]["nl"] = nl_sq
            all_seqs[name]["sql"] = sql_sq
print(f"all_seqs_num: {all_seqs_num}")
with open(f"../backend/app/data/dataset/spider/query_seq.json", "w") as f:
    json.dump(all_seqs, f)
print(len(all_seqs.keys()), all_seqs.keys())

100%|██████████| 140/140 [03:45<00:00,  1.61s/it]

all_seqs_num: 1820
140 dict_keys(['activity_1', 'aircraft', 'allergy_1', 'apartment_rentals', 'architecture', 'assets_maintenance', 'baseball_1', 'behavior_monitoring', 'bike_1', 'body_builder', 'book_2', 'browser_web', 'candidate_poll', 'chinook_1', 'cinema', 'city_record', 'climbing', 'club_1', 'coffee_shop', 'college_1', 'college_2', 'college_3', 'company_1', 'company_employee', 'company_office', 'county_public_safety', 'cre_Doc_Control_Systems', 'cre_Doc_Tracking_DB', 'cre_Docs_and_Epenses', 'cre_Drama_Workshop_Groups', 'cre_Theme_park', 'csu_1', 'culture_company', 'customer_complaints', 'customer_deliveries', 'customers_and_addresses', 'customers_and_invoices', 'customers_and_products_contacts', 'customers_campaigns_ecommerce', 'customers_card_transactions', 'debate', 'decoration_competition', 'department_management', 'department_store', 'device', 'document_management', 'dorm_1', 'driving_school', 'e_government', 'e_learning', 'election', 'election_representative', 'entertainment_

## Seq examples

1. select distinct building from faculty 
['In which buildings are there at least ten professors?'
 'Find the building that has the largest number of faculty members.'
 'Show all the distinct buildings that have faculty rooms.'
 'How many faculty members does each building have? List the result with the name of the building.'
 'Show all the buildings along with the number of faculty members the buildings have.'
 'How many faculty do we have?'
 'What is the first and last name of the faculty participating in the most activities?']
 
2. select activity_name from activity ['What activities do we have?' 'How many activities do we have?'
 'Find the number of activities available.']
 
3. select distinct rank from faculty ['What are the rank, first name, and last name of the faculty members?'
 'Which rank has the smallest number of faculty members?'
 'What are the faculty ids of all the male faculty members?'
 'What is the total number of faculty members?'
 'How many faculty members participate in an activity?'
 'What are the first and last name of the faculty who has the most students?'
 'Show all the ranks and the number of male and female faculty for each rank.'
 'Which building has most faculty members?']
 
4. select distinct rank from faculty ['For each faculty rank, show the number of faculty members who have it.'
 'How many faculty members do we have for each faculty rank?'
 'Find the faculty rank that has the least members.'
 'Show the rank, first name, and last name for all the faculty.'
 'How many faculty members do we have for each rank and gender?'
 'Show all the faculty ranks and the number of students advised by each rank.'
 'How many students are advised by each rank of faculty? List the rank and the number of students.']
 
5. select facid from faculty where sex = value ['Show the ids for all the faculty members who have at least 2 students.'
 'Show the ids of all the faculty members who participate in an activity and advise a student.'
 'Show the ids of students whose advisors are professors.'
 'Show the faculty id of each faculty member, along with the number of students he or she advises.'
 "Show ids for the faculty members who don't advise any student."
 'What are ids of the faculty members who not only participate in an activity but also advise a student.']
 
6. select count ( distinct facid ) from faculty_participates_in ['What are the faculty id and the number of students each faculty has?'
 'What is the first and last name of the faculty members who participated in at least one activity? For each of them, also show the number of activities they participated in.'
 'Show all the activity names and the number of faculty involved in each activity.'
 'Show the number of male and female assistant professors.']
 
7. select fname , lname from faculty where rank = value ['Show the first name, last name, and phone number for all female faculty members.'
 'Show the first and last name of all the faculty members who participated in some activity, together with the number of activities they participated in.'
 'Show first name and last name for all the students advised by Michael Goodrich.'
 'Give me the the first and last name of the faculty who advises the most students.'
 'Find the first and last name of the faculty who is involved in the largest number of activities.']
 
8. select stuid from student except select stuid from participates_in ['What are the ids of the students who are under 20 years old and are involved in at least one activity.'
 "Show the ids of the faculty who don't participate in any activity."
 'Show the ids for all the students who participate in an activity and are under 20.'
 'What are the ids of the faculty members who do not advise any student.']
 
9. select count ( * ) from faculty where rank = value and building = value ['How many female Professors do we have?'
 'How many male and female assistant professors do we have?'
 'Count the number of female Professors we have.'
 'Give me the number of faculty members who participate in an activity'
 'Show all the buildings that have at least 10 professors.'
 'What buildings have faculty offices?']
 
10. select fname , lname from faculty where rank = value ['What are the first name, last name, and phone number of all the female faculty members?'
 'What are the first names of the faculty members playing both Canoeing and Kayaking?'
 'What is the first and last name of the student participating in the most activities?']
 
11. select facid from faculty except select facid from faculty_participates_in ['Which faculty members advise two ore more students? Give me their faculty ids.'
 'What ranks do we have for faculty?'
 'Which students have professors as their advisors? Find their student ids.'
 'Which activity has the most faculty members participating in? Find the activity name.'
 'How many faculty members participate in each activity? Return the activity names and the number of faculty members.'
 'What are the ids of the students who are not involved in any activity'
 'What is the name of the activity that has the most faculty members involved in?'
 "Show the ids of the students who don't participate in any activity."
 'Find the first names of professors who are not playing Canoeing or Kayaking.'
 'Find the list of distinct ranks for faculty.']
 
12. select t2 . fname , t2 . lname from faculty as t1 join student as t2 on t1 . facid = t2 . advisor where t1 . fname = value and t1 . lname = value ['What are the first name and last name of all the instructors?'
 'Tell me the first and last name of the student who has the most activities.']
 
13. select distinct t1 . lname from faculty as t1 join faculty_participates_in as t2 on t1 . facid = t2 . facid join activity as t3 on t2 . actid = t2 . actid where t3 . activity_name = value or t3 . activity_name = value ['Find the ids of the students who participate in Canoeing and Kayaking.'
 'Show the first name and last name for all the instructors.'
 'Which students participate in both Canoeing and Kayaking as their activities? Tell me their student ids.']
